In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

c:\users\zhaofeng shang\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('NewsDataset_classification.csv')

In [3]:
train.shape

(39643, 52)

In [4]:
train=train.drop(['Year','shares','Date','self_reference_avg_sharess','url','timedelta','n_tokens_content'
                 ,'self_reference_min_shares','self_reference_max_shares','X'],axis=1)

In [5]:
train.head()

,Unnamed: 0,n_tokens_title,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Month,Day,weekday,isWeekend,Type,isPopular
0,1,12,0.663594,1.0,0.815385,4,2,1,0,4.680365,...,0.500000,-0.187500,0.000000,0.187500,1,7,1,0,3,Less Popular
1,2,9,0.604743,1.0,0.791946,3,1,1,0,4.913725,...,0.000000,0.000000,0.500000,0.000000,1,7,1,0,1,Less Popular
2,3,9,0.575130,1.0,0.663866,3,1,1,0,4.393365,...,0.000000,0.000000,0.500000,0.000000,1,7,1,0,1,High Popular
3,4,9,0.503788,1.0,0.665635,9,0,1,0,4.404896,...,0.000000,0.000000,0.500000,0.000000,1,7,1,0,3,Less Popular
4,5,13,0.415646,1.0,0.540890,19,19,20,0,4.682836,...,0.454545,0.136364,0.045455,0.136364,1,7,1,0,5,Less Popular


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39643 entries, 0 to 39642
Data columns (total 42 columns):
Unnamed: 0                      39643 non-null int64
n_tokens_title                  39643 non-null int64
n_unique_tokens                 39643 non-null float64
n_non_stop_words                39643 non-null float64
n_non_stop_unique_tokens        39643 non-null float64
num_hrefs                       39643 non-null int64
num_self_hrefs                  39643 non-null int64
num_imgs                        39643 non-null int64
num_videos                      39643 non-null int64
average_token_length            39643 non-null float64
num_keywords                    39643 non-null int64
kw_min_min                      39643 non-null int64
kw_max_min                      39643 non-null float64
kw_avg_min                      39643 non-null float64
kw_min_max                      39643 non-null int64
kw_max_max                      39643 non-null int64
kw_avg_max                     

In [7]:
train['Day'].describe()

count    39643.000000
mean        15.684282
std          8.637902
min          1.000000
25%          8.000000
50%         16.000000
75%         23.000000
max         31.000000
Name: Day, dtype: float64

In [8]:
def extract(data):
    data.loc[data['Day']<=8,'Day']=1
    data.loc[(data['Day']<16)&(data['Day']>8),'Day']=2
    data.loc[(data['Day']<31)&(data['Day']>=16),'Day']=2
    data.loc[data['Day']>=31,'Day']=4
    return data
train=extract(train)
    

In [9]:
x,y=train.drop(['isPopular'],axis=1),train['isPopular']

In [10]:
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size = 0.3, random_state = 100)

In [11]:
from sklearn.model_selection import GridSearchCV
def parameter_evaluate(data):
    clf = DecisionTreeClassifier()
    x,y=train.drop(['isPopular'],axis=1),train['isPopular']
    parameters={'criterion':['gini','entropy'],'min_samples_leaf':[1,2,3,4],
               'min_samples_split':[2,3,4,5]}
    grid_search = GridSearchCV(estimator=clf, param_grid=parameters, cv=10, scoring='accuracy')
    print('parameters:')
    grid_search.fit(x,y)
    print('best score:%0.3f' % grid_search.best_score_)
    print('best parameters set:')
    bsp = grid_search.best_estimator_.get_params()
    
    for i in sorted(bsp.keys()):
        print('\t%s: %s' % (i,bsp[i]))
    return bsp

parameters = parameter_evaluate(train) 

parameters:
best score:0.488
best parameters set:
	class_weight: None
	criterion: gini
	max_depth: None
	max_features: None
	max_leaf_nodes: None
	min_impurity_decrease: 0.0
	min_impurity_split: None
	min_samples_leaf: 4
	min_samples_split: 4
	min_weight_fraction_leaf: 0.0
	presort: False
	random_state: None
	splitter: best


In [12]:
cl_search = DecisionTreeClassifier(**parameters)
cl_search.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
y_pred1 = cl_search.predict(X_test)
y_pred1
print ("Accuracy is ", accuracy_score(y_test,y_pred1)*100)

Accuracy is  56.3524762465


In [ ]:

clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

In [ ]:
1
2
y_pred = clf_gini.predict(X_test)
y_pred

In [ ]:
1
print ("Accuracy is ", accuracy_score(y_test,y_pred)*100)

In [ ]:
from IPython.display import Image  
import pydotplus 
from sklearn import tree
import os  
import pydot
dot_data = tree.export_graphviz(cl_search, out_file=None, 
                         
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png()) 

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.feature_names

In [ ]:
iris.target_names


In [ ]:
train.